# Plazas Disponibles Hotelero - arbol_decision

Paso 1: Importar las librerías necesarias

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix

Paso 2: Leer los archivos Excel

In [8]:
# Leer los archivos Excel
#file_path1 = 'data1.xlsx'
#file_path2 = 'data2.xlsx'
#df1 = pd.read_excel(file_path1, sheet_name='Sheet1')
#df2 = pd.read_excel(file_path2, sheet_name='Sheet1')
datos1 = pd.read_excel("../data/external/LIMPIO_16_3_02_Oferta_Hotelera_por-sector_LIMPIO.xlsx", sheet_name="Ushuaia_1", skipfooter=2)
datos2 = pd.read_excel("../data/external/LIMPIO_16_3_02_Oferta_Hotelera_por-sector_LIMPIO.xlsx", sheet_name="Ushuaia_2", skipfooter=2)
print(datos1)

     Periodo             Mes Establecimientos Plazas disponibles  \
0       2008           enero               43              96224   
1       2008         febrero               44              87580   
2       2008           marzo               44              96069   
3       2008           abril               43              90570   
4       2008            mayo               42              90737   
..       ...             ...              ...                ...   
187     2023      agosto (*)               33             118885   
188     2023  septiembre (*)               33             115050   
189     2023      octubre(*)               33             118637   
190     2023    noviembre(*)               33             113400   
191     2023    diciembre(*)               33             115613   

    Habitaciones o Unidades disponibles  
0                                 38936  
1                                 34945  
2                                 38998  
3              

In [10]:
print(datos2)

     Periodo             Mes Establecimientos Plazas disponibles  \
0       2008           enero              104              70649   
1       2008         febrero              103              64467   
2       2008           marzo              103              68359   
3       2008           abril               94              57410   
4       2008            mayo               77              43314   
..       ...             ...              ...                ...   
187     2023      agosto (*)               37              33635   
188     2023  septiembre (*)               37              34620   
189     2023      octubre(*)               43              37975   
190     2023    noviembre(*)               42              35970   
191     2023    diciembre(*)               42              39742   

    Habitaciones o Unidades disponibles  
0                                 21452  
1                                 19372  
2                                 20864  
3              

Paso 3: Combinar los DataFrames

In [13]:
# Combinar los DataFrames
df = pd.concat([datos1, datos2], ignore_index=True)
print(df)

     Periodo             Mes Establecimientos Plazas disponibles  \
0       2008           enero               43              96224   
1       2008         febrero               44              87580   
2       2008           marzo               44              96069   
3       2008           abril               43              90570   
4       2008            mayo               42              90737   
..       ...             ...              ...                ...   
379     2023      agosto (*)               37              33635   
380     2023  septiembre (*)               37              34620   
381     2023      octubre(*)               43              37975   
382     2023    noviembre(*)               42              35970   
383     2023    diciembre(*)               42              39742   

    Habitaciones o Unidades disponibles  
0                                 38936  
1                                 34945  
2                                 38998  
3              

Paso 4: Preprocesar los datos
Convertir columnas a tipo numérico


In [16]:
df['Plazas disponibles'] = pd.to_numeric(df['Plazas disponibles'], errors='coerce')
df['Habitaciones o Unidades disponibles'] = pd.to_numeric(df['Habitaciones o Unidades disponibles'], errors='coerce')
df.dropna(inplace=True)
print(df)



     Periodo             Mes Establecimientos  Plazas disponibles  \
0       2008           enero               43             96224.0   
1       2008         febrero               44             87580.0   
2       2008           marzo               44             96069.0   
3       2008           abril               43             90570.0   
4       2008            mayo               42             90737.0   
..       ...             ...              ...                 ...   
379     2023      agosto (*)               37             33635.0   
380     2023  septiembre (*)               37             34620.0   
381     2023      octubre(*)               43             37975.0   
382     2023    noviembre(*)               42             35970.0   
383     2023    diciembre(*)               42             39742.0   

     Habitaciones o Unidades disponibles  
0                                38936.0  
1                                34945.0  
2                                38998.0  

Convertir la columna de mes a un formato adecuado

In [19]:
#Crear un diccionario para convertir los nombres de los meses a números
month_map = {
    'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
    'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12
}
df['Mes_num'] = df['Mes'].map(month_map)
print(df)

     Periodo             Mes Establecimientos  Plazas disponibles  \
0       2008           enero               43             96224.0   
1       2008         febrero               44             87580.0   
2       2008           marzo               44             96069.0   
3       2008           abril               43             90570.0   
4       2008            mayo               42             90737.0   
..       ...             ...              ...                 ...   
379     2023      agosto (*)               37             33635.0   
380     2023  septiembre (*)               37             34620.0   
381     2023      octubre(*)               43             37975.0   
382     2023    noviembre(*)               42             35970.0   
383     2023    diciembre(*)               42             39742.0   

     Habitaciones o Unidades disponibles  Mes_num  
0                                38936.0      NaN  
1                                34945.0      NaN  
2              

Codificar variables categóricas

In [22]:
label_encoders = {}
for column in ['Mes', 'Establecimientos']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le


In [24]:
# Eliminar filas con valores nulos
df.dropna(inplace=True)

In [26]:
# Verificar que no hay valores nulos en el conjunto de datos
print(df.isnull().sum())

Periodo                                0.0
Mes                                    0.0
Establecimientos                       0.0
Plazas disponibles                     0.0
Habitaciones o Unidades disponibles    0.0
Mes_num                                0.0
dtype: float64


Paso 5: Definir las variables independientes y dependientes

In [29]:
X = df[['Mes_num', 'Establecimientos', 'Habitaciones o Unidades disponibles']]
y = df['Plazas disponibles']

Binarizar la variable dependiente

In [32]:
y = (y > y.median()).astype(int)


In [34]:
# Verificar si hay suficientes muestras para cada clase después de binarizar
print(y.value_counts())

Series([], Name: Plazas disponibles, dtype: int64)


In [36]:
# Verificar si hay suficientes datos para dividir en conjuntos de entrenamiento y prueba
if len(X) < 2 or len(X) < 2:
    raise ValueError("No hay suficientes datos para dividir en conjuntos de entrenamiento y prueba.")

ValueError: No hay suficientes datos para dividir en conjuntos de entrenamiento y prueba.

Paso 6: Dividir los datos en conjunto de entrenamiento y prueba

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



Paso 7: Entrenar el modelo de árbol de decisión

In [ ]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)


Paso 8: Evaluar el modelo

In [ ]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Paso 9: Visualizar el árbol de decisión

In [ ]:
plt.figure(figsize=(20,10))
plot_tree(model, feature_names=X.columns, class_names=['Bajo', 'Alto'], filled=True)
plt.title('Árbol de decisión para la oferta de plazas disponibles en Ushuaia')
plt.show()
